In [5]:
import SimpleITK as sitk
import pandas as pd
import time
import datetime
import numpy as np

numero_de_interacoes = []
valor_final_da_metrica = []
tempo_incial_codigo = time.time()

def configuracao_joint_histogram_mutual_information(metodo_de_registro):
    metodo_de_registro.SetMetricAsJointHistogramMutualInformation(numberOfHistogramBins=20)
    metodo_de_registro.SetMetricSamplingStrategy(metodo_de_registro.REGULAR)
    metodo_de_registro.SetMetricSamplingPercentage(1.0)
    return metodo_de_registro

def configuracao_mattes_mutual_information(metodo_de_registro):
    metodo_de_registro.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    metodo_de_registro.SetMetricSamplingStrategy(metodo_de_registro.RANDOM)
    metodo_de_registro.SetMetricSamplingPercentage(0.2)
    return metodo_de_registro

def configuracao_correlation(metodo_de_registro):
    metodo_de_registro.SetMetricAsCorrelation()
    metodo_de_registro.SetMetricSamplingStrategy(metodo_de_registro.REGULAR)
    metodo_de_registro.SetMetricSamplingPercentage(1.0)
    return metodo_de_registro

def configuracao_ants_neighborhood_correlation(metodo_de_registro):
    metodo_de_registro.SetMetricAsANTSNeighborhoodCorrelation(radius=3)
    metodo_de_registro.SetMetricSamplingStrategy(metodo_de_registro.RANDOM)
    metodo_de_registro.SetMetricSamplingPercentage(0.2)
    return metodo_de_registro

def calcular_translacao(imagem_drr, imagem_cr, funcao_de_configuracao):
    
    if imagem_drr.GetNumberOfComponentsPerPixel() > 1:
        imagem_drr = sitk.VectorIndexSelectionCast(imagem_drr, 0)
    if imagem_cr.GetNumberOfComponentsPerPixel() > 1:
        imagem_cr = sitk.VectorIndexSelectionCast(imagem_cr, 0)

    #inicializacao
    transformacao_inicial = sitk.TranslationTransform(2)
    
    #parametro iniciais de registro definidos pelo usuario
    deslocamento_inicial_x = 10
    deslocamento_inicial_y = 10
    
    if deslocamento_inicial_x == 0 or deslocamento_inicial_y == 0:
        transformacao_inicial = sitk.CenteredTransformInitializer(imagem_drr,
                                                              imagem_cr,sitk.Euler2DTransform(),
                                                              sitk.CenteredTransformInitializerFilter.GEOMETRY)
    else:
        transformacao_inicial.SetOffset((deslocamento_inicial_x, deslocamento_inicial_y))
    
    #Configure a métrica usando a função de configuração fornecida
    metodo_de_registro = sitk.ImageRegistrationMethod()
    metodo_de_registro = funcao_de_configuracao(metodo_de_registro)
    metodo_de_registro.SetMetricSamplingStrategy(metodo_de_registro.RANDOM)
    metodo_de_registro.SetMetricSamplingPercentage(0.20)
    
    # Configurações do otimizador
    metodo_de_registro.SetOptimizerAsGradientDescent(learningRate=1.0, 
                                                      numberOfIterations=50, 
                                                      convergenceMinimumValue=1e-6, 
                                                      convergenceWindowSize=20)
    metodo_de_registro.SetOptimizerScalesFromPhysicalShift()
    
    # Configuração para o framework de múltiplas resoluções
    metodo_de_registro.SetShrinkFactorsPerLevel(shrinkFactors=[4, 2, 1])
    metodo_de_registro.SetSmoothingSigmasPerLevel(smoothingSigmas=[2, 1, 0])
    metodo_de_registro.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    # reexecutar o registro várias vezes sem se preocupar em redefinir a transformação inicial, pois ela não será alterada
    metodo_de_registro.SetInitialTransform(transformacao_inicial, inPlace=False)

    tempo_incial = time.time()
    transformacao_final = metodo_de_registro.Execute(sitk.Cast(imagem_drr, sitk.sitkFloat32),
                                                 sitk.Cast(imagem_cr, sitk.sitkFloat32))
    tempo_final = time.time()
    tempo_total_registro = tempo_final - tempo_incial
    valor_final_da_metrica = metodo_de_registro.GetMetricValue()
    numero_de_interacoes = metodo_de_registro.GetOptimizerIteration()
    if isinstance(transformacao_final, sitk.CompositeTransform):
        parametros_finais = transformacao_final.GetNthTransform(transformacao_final.GetNumberOfTransforms() - 1).GetParameters()
    else:
        print(f'Tipo de transformação inesperado: {type(transformacao_final)}')
        return None, None, None, None
    print("parametros_finais: ",parametros_finais)
    return parametros_finais, valor_final_da_metrica, tempo_total_registro, numero_de_interacoes, deslocamento_inicial_x, deslocamento_inicial_y

funcao_de_configuracaos = {
    'JointHistogramMutualInformation': configuracao_joint_histogram_mutual_information,
    'MattesMutualInformation': configuracao_mattes_mutual_information,
    'Correlation': configuracao_correlation,
    'ANTSNeighborhoodCorrelation': configuracao_ants_neighborhood_correlation,
}

par_de_imagens = [
    #("Par1_dcm/DRR.0.png", "Par1_dcm/CR.0.png"),
    #("Par1_dcm/DRR.270.png", "Par1_dcm/CR.270.png"),
    #("Par2_dcm/DRR.0.png", "Par2_dcm/CR.0.png"),
    #("Par2_dcm/DRR.270.png", "Par2_dcm/CR.270.png"),
    #("Par3_dcm/DRR.0.png", "Par3_dcm/CR.0.png"),
    ("Par3_dcm/DRR.270.png", "Par3_dcm/CR.270.png"),
]

resultados_do_registro = []
valor_deslocamento = {'X': {}, 'Y': {}}
numero_maximo_de_tentativas = 5
numero_de_interacao_de_registro = 2

for drr_caminho, cr_caminho in par_de_imagens:
    for interacao in range(numero_de_interacao_de_registro):
        print(f'Iteração {interacao + 1} - Processando imagens {drr_caminho} e {cr_caminho}')
        imagem_fixa = sitk.ReadImage(drr_caminho)
        imagem_em_movimento = sitk.ReadImage(cr_caminho)
        for nome_da_metrica, funcao_de_configuracao in funcao_de_configuracaos.items():
            tentativas = 0
            while tentativas < numero_maximo_de_tentativas:
                print(f'Rodando com a métrica {nome_da_metrica} - Tentativa {tentativas + 1}')
                parametros_finais, valor_final_da_metrica, tempo_total_registro, numero_de_interacoes, deslocamento_inicial_x, deslocamento_inicial_y = calcular_translacao(imagem_fixa, imagem_em_movimento, funcao_de_configuracao)
                
                if parametros_finais is not None:
                    vetor_de_deslocamento = np.array(parametros_finais)
                    # Verifica se o deslocamento está dentro do intervalo esperado
                    if -60 <= vetor_de_deslocamento[0] <= 60 and -60 <= vetor_de_deslocamento[1] <= 60:
                        break
                    else:
                        tentativas += 1
                        continue

            if tentativas == numero_maximo_de_tentativas:
                print(f"Não foi possível alcançar um deslocamento aceitável após {numero_maximo_de_tentativas} tentativas com a métrica {nome_da_metrica}.")
                continue

            for id_eixo, eixo in enumerate(['X', 'Y']):
                if nome_da_metrica not in valor_deslocamento[eixo]:
                    valor_deslocamento[eixo][nome_da_metrica] = []
                valor_deslocamento[eixo][nome_da_metrica].append(vetor_de_deslocamento[id_eixo])

            resultados_do_registro.append({
                'imagem_fixa': drr_caminho,
                'imagem_em_movimento': cr_caminho,
                'nome_da_metrica': nome_da_metrica,
                'vetor_de_deslocamento': vetor_de_deslocamento,
                'valor_final_da_metrica': valor_final_da_metrica,
                'tempo_total_registro': tempo_total_registro,
                'numero_de_interacoes': numero_de_interacoes,
                'deslocamento_inicial_x':deslocamento_inicial_x,
                'deslocamento_inicial_y':deslocamento_inicial_y
            })
            print("vetor_de_deslocamento:",vetor_de_deslocamento)
        else:
            print('Falha ao computar a translação')

# Salvando o DataFrame em um arquivo CSV
df = pd.DataFrame(resultados_do_registro)
# Formata a data e hora no formato desejado (dd.MM.yyyy.HHmm)
horario_atual = datetime.datetime.now()
tupla = par_de_imagens[0]
nome_da_imagem_modificado = tupla[0].replace("/", ".").replace("_dcm", "").replace(".png", "")
filename = f"{nome_da_imagem_modificado}.{horario_atual.strftime('%d.%m.%Y.%H%M')}.csv"
df.to_csv(filename, sep='\t', index=True)  

# Calculo do tempo total da simulação de registro de imagem
tempo_final_codigo = time.time()
tempo_total_registro = tempo_final_codigo - tempo_incial_codigo
print("Tempo total[s]:", tempo_total_registro)

Iteração 1 - Processando imagens Par3_dcm/DRR.270.png e Par3_dcm/CR.270.png
Rodando com a métrica JointHistogramMutualInformation - Tentativa 1
parametros_finais:  (38.00417108557276, 28.356603231109013)
vetor_de_deslocamento: [38.00417109 28.35660323]
Rodando com a métrica MattesMutualInformation - Tentativa 1
parametros_finais:  (0.9692544506893341, 34.68584057047039)
vetor_de_deslocamento: [ 0.96925445 34.68584057]
Rodando com a métrica Correlation - Tentativa 1
parametros_finais:  (-169.5288787741258, 147.9890171771869)
Rodando com a métrica Correlation - Tentativa 2
parametros_finais:  (32.52992742801378, 24.828545515862388)
vetor_de_deslocamento: [32.52992743 24.82854552]
Rodando com a métrica ANTSNeighborhoodCorrelation - Tentativa 1
parametros_finais:  (43.21273488082327, 7.667288791453051)
vetor_de_deslocamento: [43.21273488  7.66728879]
Falha ao computar a translação
Iteração 2 - Processando imagens Par3_dcm/DRR.270.png e Par3_dcm/CR.270.png
Rodando com a métrica JointHistogr